In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'omkar_proj.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
from account.models import (
    User,
    Farmer,
    Rental,
    Investor,
)
from warehouse.models import (
    Gala,
    Property,
    Company,
)
from django_pandas.io import read_frame
from django.db.models import Count,F,Sum,OuterRef

In [3]:

# # rf = Gala.objects.filter(is_allotted=True).values('uid','gala_number'
# #                                                   ).annotate(
# #     total_not_allotted_galas = Count('warehouse__uid'))
# # rf
# # rf = Property.objects.filter(company__name__exact='OMKAR').values('uid','property_name'
# #                                     ).annotate(get_galas = Gala.objects.filter(is_allotted=False))

# rf = Property.objects.filter(company__name__exact='OMKAR').values('uid','property_name'
#                                                                  ).annotate(get_galas= Property.objects.prefetch_related('get_gala').filter(get_gala__is_allotted=False))



In [4]:
rf = Company.objects.get(name='OMKAR').get_properties.all().values('uid','property_name'
            ).annotate(total_remaining_galas =Gala.objects.filter(warehouse__uid=OuterRef('uid'),is_allotted=False
                                                                 ).values('warehouse__uid'
                    ).annotate(total_remaining_galas=Count('id')).values('total_remaining_galas'))
pd = read_frame(rf)
pd
                               


,uid,property_name,total_remaining_galas
0,2bfbb94a-b39c-4365-ae5e-f14b5b9339d6,warehouse 4,1
1,5d3288a4-47a6-411e-a821-b8004829aec1,warehouse 3,1


In [5]:
get_company=Company.objects.get(name__iexact='OMKAR').get_properties.all(
                ).values("uid","property_name","property_survey_number",
                         "property_type","address","city","zipcode","country","state","company__name"
                        ).annotate(total_remaining_galas= Gala.objects.filter(warehouse__uid=OuterRef('uid'),is_allotted=False
                                        ).values('warehouse__uid').annotate(total_remaining_galas=Count('id')).values('total_remaining_galas'))



#.get_gala.select_related("get_gala_detail").values("uid","gala_number","is_allotted"
#                     ).annotate(total_number_of_galas=Gala.objects.filter(uid))
rf = read_frame(get_company)
rf

,uid,property_name,property_survey_number,property_type,address,city,zipcode,country,state,company__name,total_remaining_galas
0,2bfbb94a-b39c-4365-ae5e-f14b5b9339d6,warehouse 4,ae8ff192-1b22-4ffb-8494-42e43375fdf3,RCC,M.G Road,dewas,455115,India,M.P,OMKAR,1
1,5d3288a4-47a6-411e-a821-b8004829aec1,warehouse 3,4a5e1400-b07f-48d0-93b9-e8fafdfe4f13,COLD STORAGE,M.G Road,dewas,455115,India,M.P,OMKAR,1


In [19]:
from django.db.models import Prefetch

In [22]:
get_remaining_galas = Company.objects.get(name__iexact='OMKAR').get_properties.all().prefetch_related(
    Prefetch(
        "get_gala",
        queryset = Gala.objects.filter(is_allotted = False)
    )
)

In [23]:
get_remaining_galas

<QuerySet [<Property: warehouse 4>, <Property: warehouse 3>]>